# Set up config variables

In [1]:
import json

In [2]:
path = ""
with open(path, 'r') as f:
        regression_mesh_config = json.load(f)

config = regression_mesh_config['config']

In [3]:
latMin = config['Mesh_info']['Region']['latMin']  
latMax = config['Mesh_info']['Region']['latMax'] 
longMin = config['Mesh_info']['Region']['longMin']
longMax = config['Mesh_info']['Region']['longMax']
startTime = config['Mesh_info']['Region']['startTime']
endTime = config['Mesh_info']['Region']['endTime']
cellWidth = config['Mesh_info']['Region']['cellWidth']
cellHeight = config['Mesh_info']['Region']['cellHeight']

config

{'Mesh_info': {'Region': {'latMin': -65,
   'latMax': -60,
   'longMin': -70,
   'longMax': -50,
   'startTime': '2019-01-01',
   'endTime': '2019-01-14',
   'cellWidth': 5,
   'cellHeight': 2.5}}}

### Generate empty mesh, just with config data

In [4]:
from polar_route.mesh import Mesh
import polar_route.data_loaders as data_loader

regression_mesh = Mesh(config)

ImportError: dlopen(/Users/ayat/BAS/bas-venv/lib/python3.10/site-packages/pyproj/_network.cpython-310-darwin.so, 2): Symbol not found: ____chkstk_darwin
  Referenced from: /Users/ayat/BAS/bas-venv/lib/python3.10/site-packages/pyproj/.dylibs/libtiff.5.dylib (which was built for Mac OS X 11.0)
  Expected in: /usr/lib/libSystem.B.dylib
 in /Users/ayat/BAS/bas-venv/lib/python3.10/site-packages/pyproj/.dylibs/libtiff.5.dylib

# Use Data Loaders for each data type

In [6]:
gebco_loader = getattr(data_loader, "load_gebco")
data_source = config['Mesh_info']['Data_sources'][0]
params = data_source['params']
#params['file'] = '../../../BAS/PolarRoute/datastore/bathymetry/GEBCO/gebco_2022_n-40.0_s-90.0_w-140.0_e0.0.nc'


bathymetry_datapoints = gebco_loader(params, longMin,longMax,latMin,latMax,startTime,endTime)

NameError: name 'data_loader' is not defined

In [1]:
bathymetry_datapoints[0]

NameError: name 'bathymetry_datapoints' is not defined

### Add datapoints to the mesh

In [2]:

regression_mesh.add_data_points(bathymetry_datapoints)

NameError: name 'demo_mesh' is not defined

# Adjust methods of binning

Can be mean, max, min, std, etc... (some not implemented). Default is mean

e.g. For each cell, mean elevation is not as important as maximum elevation (ship could crash if not using max), so set MAX to be the output type for each cellbox

In [3]:
value_output_type = '{"elevation": "MAX"}'
value_output_type = json.loads(value_output_type)

NameError: name 'json' is not defined

### Set data source if no datapoints remain after splitting

Can also be set to static value

e.g. If cellbox is split, and no datapoints remain, will default to NaN, which breaks code. 'parent' sets value to be same as parent cellbox if splitting and no datapoints remain. 

In [4]:
value_fill_type = '{}'
value_fill_type = json.loads(value_fill_type)
value_fill_type['SIC'] = 'parent'
value_fill_type['uC'] = 'parent'
value_fill_type['vC'] = 'parent'
value_fill_type['thickness'] = 'parent'
value_fill_type['density'] = 'parent'
value_fill_type

NameError: name 'json' is not defined

# Set cellbox behaviour metadata

5Setting output_type and fill_type first, as splitting operations will depend on these

In [5]:
from polar_route.cellbox import CellBox

for cellbox in regression_mesh.cellboxes:
    if isinstance(cellbox, CellBox):
        cellbox.add_value_output_type(value_output_type)
        cellbox.set_value_fill_types(value_fill_type)
        

ModuleNotFoundError: No module named 'polar_route'

### Split based on each variable's extreme conditions

In [6]:
splitting_condition = config['Mesh_info']['splitting']['splitting_conditions'][0]
splitting_condition = json.loads(splitting_condition)


for cellbox in regression_mesh.cellboxes:
    if isinstance(cellbox, CellBox):
        cellbox.add_splitting_condition(splitting_condition)

split_depth =  config['Mesh_info']['splitting']['split_depth']       
regression_mesh.split_to_depth(4)

NameError: name 'json' is not defined

# Generate output from cellbox creation

Saved as JSON

In [8]:
demo_output = regression_mesh.to_json()

NameError: name 'demo_mesh' is not defined

# Plot

Read in JSON output, plot each layer by type